## Setup

In [107]:
import pandas as pd
import numpy as np
import json
import glob
import matplotlib.pyplot as plt
%matplotlib inline

pd.options.display.max_rows = 999
pd.options.display.max_columns=100
plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = [15, 10]

dataDir = '/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/'
jsonDir = '/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/JSONFiles/'

## Read and fix data

In [29]:
utils = pd.read_csv(dataDir + 'eia_data/eia8602018/utility_2018.csv',header=1).fillna(0)
utils.columns = [x.lower().replace(" ","_").replace("?","") for x in utils.columns]

In [30]:
utils['owner_of_plants_reported_on_form'].value_counts()

Y    4400
0     639
Name: owner_of_plants_reported_on_form, dtype: int64

In [31]:
utils[utils['owner_of_plants_reported_on_form']==0];

In [32]:
gen_data = pd.read_csv(dataDir + 'eia_data/eia8602018/gen_2018.csv')
gen_data.drop(['Unnamed: 0'],axis=1,inplace=True)
gen_data = gen_data[~gen_data['Utility ID'].str.contains('NOTE', na=False)]
gen_data['Utility ID'] = pd.to_numeric(gen_data['Utility ID'])
gen_data.columns = gen_data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('?','')

Now I want to merge the utility data with the generator data so I can do some sick analyses bro

I'll start by merging the utility list with the list of generators (all in 2018), so I'll see if any columns will conflict

In [33]:
[print(x) for x in gen_data.columns if x in utils.columns]

utility_id
utility_name
state


[None, None, None]

I'm merging by utility_id so let's fix the state variable

In [34]:
gen_data = gen_data.rename(columns={'state': 'plant_state'})

In [35]:
merged = gen_data.merge(utils, how='left', on=['utility_id','utility_name'])

Various checks assures me that the merge was successful. So now I want to merge in ownership data in a clever way

## Now merge in the owner variable in a creative way
### solution is to add owner_1 owner_1_perc, owner_2 ... etc.  for each plant

In [36]:
# S Single ownership by respondent 
# J Jointly owned with another entity
# W Wholly owned by an entity other than respondent
df = merged.join(pd.get_dummies(merged['ownership']))
df.rename(columns={'S': 'single_owner', 'W': 'other_owner', 'J':'joint_owner'},inplace=True)

In [37]:
owners = pd.read_csv(dataDir + 'eia_data/eia8602018/owner_2018.csv',header=1)
owners.columns = owners.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('?','')

owners['perc_owned'] = pd.to_numeric(owners['percent_owned'].str.replace("%",""))

num_owners = owners.groupby(['plant_name', 'generator_id']).apply(lambda grp: len(grp.owner_name.unique())).reset_index()
num_owners.rename(columns={0: 'num_owners'},inplace=True)
owners = owners.merge(num_owners, how='outer', on=['plant_name','generator_id'])

In [38]:
owners['idx'] = owners['plant_code'].astype(str) + '_' + owners['generator_id']

In [39]:
# owner_dict with key = generator; value = owners 
owner_dict = owners.groupby(['idx']).owner_name.apply(lambda grp: list(grp.value_counts().index)).to_dict()

perc_dict = owners.groupby(['idx']).perc_owned.apply(lambda grp: list(grp)).to_dict()

# dictionary may be useful for other stuff 
perc_owned_dict = {}
for x in owner_dict:
    temp_dict = {}
    for i,y in enumerate(owner_dict[x]):
        temp_dict.update({y : perc_dict[x][i]})
    perc_owned_dict[x] = temp_dict

In [40]:
owners = owners.join(owners['idx'].map(owner_dict).apply(pd.Series).iloc[:,0:4].rename(columns={0: 'owner_1', 1: 'owner_2', 2: 'owner_3', 3: 'owner_4'}))

owners = owners.join(owners['idx'].map(perc_dict).apply(pd.Series).iloc[:,0:4].rename(columns={0: 'perc_owner_1', 1: 'perc_owner_2', 2: 'perc_owner_3', 3: 'perc_owner_4'}))


## Apply parent dict to the list of owners

In [41]:
with open(jsonDir + 'final_parent_dict.json') as outfile:
    parent_map = json.load(outfile)

In [42]:
owners['parent_1'] = owners.owner_1.str.lower().map(parent_map)
owners['parent_2'] = owners.owner_2.str.lower().map(parent_map)
owners['parent_3'] = owners.owner_3.str.lower().map(parent_map)
owners['parent_4'] = owners.owner_4.str.lower().map(parent_map)

In [43]:
owners['diff_parent'] = owners['parent_1'] != owners['owner_1']

## Drop duplicates before merging

In [44]:
# list of cols to ignore to avoid _x and _y 
cols_to_ignore = ['utility_id',
 'utility_name',
 'plant_name',
 'state',
 'status'] 

owners_to_merge = owners.drop_duplicates('idx')
owners_to_merge = owners_to_merge.drop([x for x in df.columns if x in owners.columns if x in cols_to_ignore],axis=1)
df = df.merge(owners_to_merge, how='left', on=['plant_code','generator_id'])

In [45]:
df.to_csv(dataDir + 'master_gen_by_owner_2028.csv')

## Merge owners back into generator data

### Now do the parent utility stuff

In [46]:
# fill na of parent 1 with parent map on the utility name

df['parent_1'] = df['parent_1'].fillna(df.utility_name.str.lower().map(parent_map))

In [47]:
df['nameplate_cap']=pd.to_numeric(df['nameplate_capacity_mw'].astype(str).str.replace(",",""))

In [48]:
solar = df[df['technology']=='Solar Photovoltaic']
#wind=df[df['energy_source_1']=='WND']

In [49]:
decode_parent_map = {
    95: 'nextera',
    5: 'southern current',
    63: 'tenaska capital',
    137: 'exelon',
    1248: 'consolidated edison',
    1442: 'dominion',
    1348: 'cypress creek renewables',
    128: 'southern power co',
    227: 'spower',
    10: '8me',
    320: 'berkshire hathaway',
    3363: 'solar star',
    1694: 'first solar asset management',
    2653: 'sunedison',
    1465: 'nrg',
    363: 'nrg',
    6: '174 power',
    1230: 'vistra',
    566: 'arevon',
    3084: 'canadian solar',
    3403: 'southern power co',
    612: 'duke energy',
    1695: 'first solar project development'
    
    
}

In [50]:
solar['parent_1_name']=solar.parent_1.map(decode_parent_map).fillna(solar['parent_1'])
solar['parent_2_name']=solar.parent_2.map(decode_parent_map).fillna(solar['parent_2'])

/Users/rachelanderson/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/rachelanderson/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [119]:
pd.set_option('display.max_rows', None)
df = solar.groupby(['plant_state','operating_year']).nameplate_cap.sum().sort_values(ascending=False).reset_index()
df.sort_values(by='operating_year')

,plant_state,operating_year,nameplate_cap
211,AZ,2001.0,3.9
233,AZ,2002.0,2.1
273,AZ,2003.0,0.1
272,CA,2003.0,0.2
245,CA,2005.0,1.6
270,NJ,2005.0,0.5
266,NJ,2006.0,0.9
263,CA,2006.0,1.0
248,NJ,2007.0,1.3
144,NV,2007.0,15.0


In [77]:
nextera_solar=solar[(solar['parent_1']==95) | (solar['parent_2']==95)].sort_values(by='operating_year')[['operating_year','plant_state','plant_name','utility_name','nameplate_cap']]

In [60]:
proposed_gen = pd.read_csv("/Users/rachelanderson/Dropbox (Princeton)/Tax Equity Code/Clean Data/proposed_gen_master_list_post08.csv")
prop_solar=proposed_gen[proposed_gen['technology']=="Solar Photovoltaic"];

In [66]:
prop_solar['parent_id']=prop_solar.utility_name.str.lower().map(parent_map)
prop_solar['parent_name']=prop_solar['parent_id'].map(decode_parent_map)

/Users/rachelanderson/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/rachelanderson/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [78]:
nextera_solar

,operating_year,plant_state,plant_name,utility_name,nameplate_cap
15560,2009.0,FL,DeSoto Next Generation Solar Energy,Florida Power & Light Co,25.0
15561,2010.0,FL,Space Coast Next Gen Solar Energy,Florida Power & Light Co,10.0
16612,2011.0,NM,"Hatch Solar Energy Center I, LLC","Hatch Solar Energy Center I, LLC",5.0
18048,2011.0,NJ,Paradise Solar Energy Center,"Paradise Solar Urban Renewal, LLC",5.0
18015,2014.0,NV,Mountain View Solar,NextEra Energy Mountain View Solar,20.0
20116,2015.0,MA,Hardwick-Athol & Eagle Hill,"SEC CRSD Solar One, LLC",2.8
20114,2015.0,NY,Oneida - West,"Oneida DG Solar, LLC",1.9
20113,2015.0,NY,Oneida - South,"Oneida DG Solar, LLC",1.9
19064,2015.0,CA,"Adelanto Solar, LLC","Adelanto Solar, LLC",20.0
19063,2015.0,CA,"Adelanto Solar II, LLC","Adelanto Solar II, LLC",7.0


In [84]:
nextera_prop_solar = prop_solar[prop_solar.plant_name.isin(nextera_plants['plant_name'])][['plant_name','plant_state','scheduled_start_year','scheduled_start_month','curr_scheduled_year','curr_sceheduled_month','year']]

In [95]:
nextera_solar.to_csv(dataDir + 'company_data/NEP_to_check.csv')

In [101]:
solar[solar.plant_name.str.contains('Blue')].sort_values(by='plant_state')

,utility_id,utility_name,plant_code,plant_name,plant_state,county,generator_id,technology,prime_mover,unit_code,ownership,duct_burners,can_bypass_heat_recovery_steam_generator,rto/iso_lmp_node_designation,rto/iso_location_designation_for_reporting_wholesale_sales_data_to_ferc,nameplate_capacity_mw,nameplate_power_factor,summer_capacity_mw,winter_capacity_mw,minimum_load_mw,uprate_or_derate_completed_during_year,month_uprate_or_derate_completed,year_uprate_or_derate_completed,status,synchronized_to_transmission_grid,operating_month,operating_year,planned_retirement_month,planned_retirement_year,associated_with_combined_heat_and_power_system,sector_name,sector,topping_or_bottoming,energy_source_1,energy_source_2,energy_source_3,energy_source_4,energy_source_5,energy_source_6,startup_source_1,startup_source_2,startup_source_3,startup_source_4,solid_fuel_gasification_system,carbon_capture_technology,turbines_or_hydrokinetic_buoys,time_from_cold_shutdown_to_full_load,fluidized_bed_technology,pulverized_coal_technology,stoker_technology,...,planned_derate_year,planned_new_prime_mover,planned_energy_source_1,planned_new_nameplate_capacity_mw,planned_repower_month,planned_repower_year,other_planned_modifications,other_modifications_month,other_modifications_year,multiple_fuels,cofire_fuels,switch_between_oil_and_natural_gas,street_address,city,state,zip,owner_of_plants_reported_on_form,operator_of_plants_reported_on_form,asset_manager_of_plants_reported_on_form,other_relationships_with_plants_reported_on_form,entity_type,joint_owner,single_owner,other_owner,owner_name,owner_street_address,owner_city,owner_state,owner_zip,ownership_id,percent_owned,perc_owned,num_owners,idx,owner_1,owner_2,owner_3,owner_4,perc_owner_1,perc_owner_2,perc_owner_3,perc_owner_4,parent_1,parent_2,parent_3,parent_4,diff_parent,nameplate_cap,parent_1_name,parent_2_name
18124,58661,"Sustainable Power Group, LLC",58626.0,Western Antelope Blue Sky Ranch A,CA,Los Angeles,WABSA,Solar Photovoltaic,PV,NaN,S,X,X,NaN,NaN,20,,19.0,18.0,,N,,,OP,X,11.0,2014.0,,,N,IPP Non-CHP,2.0,X,SUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,,NaN,NaN,NaN,NaN,...,,NaN,NaN,,,,NaN,,,NaN,NaN,NaN,"2180 South 1300 East, Suite 60",Salt Lake City,UT,84106,Y,0,0,0,Q,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,227.0,NaN,NaN,NaN,NaN,20.0,spower,NaN
19640,58661,"Sustainable Power Group, LLC",59961.0,Western Antelope Blue Sky B,CA,Los Angeles,WABSB,Solar Photovoltaic,PV,NaN,W,X,X,NaN,NaN,20,,19.0,19.0,,N,,,OP,X,12.0,2016.0,,,N,IPP Non-CHP,2.0,X,SUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,,NaN,NaN,NaN,NaN,...,,NaN,NaN,,,,NaN,,,NaN,NaN,NaN,"2180 South 1300 East, Suite 60",Salt Lake City,UT,84106,Y,0,0,0,Q,0,0,1,Western Antelope Blue Sky Ranch B,2180 South 1300 East Ste 600,Salt Lake City,UT,84106,59719.0,100.00%,100.0,1.0,59961_WABSB,Western Antelope Blue Sky Ranch B,NaN,NaN,NaN,100.0,NaN,NaN,NaN,2747.0,NaN,NaN,NaN,True,20.0,2747,NaN
20717,6452,Florida Power & Light Co,61029.0,Blue Cypress Solar Energy Center,FL,Indian River,1,Solar Photovoltaic,PV,NaN,S,X,X,NaN,NaN,74.5,,74.9,74.9,,N,,,OP,X,3.0,2018.0,,,N,Electric Utility,1.0,X,SUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,...,,NaN,NaN,,,,NaN,,,N,NaN,NaN,P O Box 29100,Miami,FL,33102,Y,0,0,0,I,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0,NaN,NaN,NaN,NaN,74.5,nextera,NaN
19402,59519,"TerraForm Solar XVII, LLC",59752.0,BlueWave Capital - Grafton (SREC II),MA,Worcester,1,Solar Photovoltaic,PV,NaN,S,X,X,NaN,NaN,1.3,,1.3,1.3,,N,,,OP,X,3.0,2015.0,,,N,IPP Non-CHP,2.0,X,SUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,...,,NaN,NaN,,,,NaN,,,NaN,NaN,NaN,12500 Baltimore Avenue,Beltsville,MD,20705,Y,0,0,0,Q,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,306.0,NaN,NaN,NaN,NaN,1.3,306,NaN
19348,59483,Metropolitan Airports Commission,59709.0,St. Paul Intl Airport Red & Blue Parking,MN,Hennepin,PV1,Solar Photovoltaic,PV,NaN,S,X,X,NaN,

In [91]:
nextera_solar[~nextera_solar.plant_name.isin(nextera_prop_solar)]

,operating_year,plant_state,plant_name,utility_name,nameplate_cap
15560,2009.0,FL,DeSoto Next Generation Solar Energy,Florida Power & Light Co,25.0
15561,2010.0,FL,Space Coast Next Gen Solar Energy,Florida Power & Light Co,10.0
16612,2011.0,NM,"Hatch Solar Energy Center I, LLC","Hatch Solar Energy Center I, LLC",5.0
18048,2011.0,NJ,Paradise Solar Energy Center,"Paradise Solar Urban Renewal, LLC",5.0
18015,2014.0,NV,Mountain View Solar,NextEra Energy Mountain View Solar,20.0
20116,2015.0,MA,Hardwick-Athol & Eagle Hill,"SEC CRSD Solar One, LLC",2.8
20114,2015.0,NY,Oneida - West,"Oneida DG Solar, LLC",1.9
20113,2015.0,NY,Oneida - South,"Oneida DG Solar, LLC",1.9
19064,2015.0,CA,"Adelanto Solar, LLC","Adelanto Solar, LLC",20.0
19063,2015.0,CA,"Adelanto Solar II, LLC","Adelanto Solar II, LLC",7.0


In [86]:
nextera_prop_solar.drop_duplicates().sort_values(by='curr_scheduled_year')

,plant_name,plant_state,scheduled_start_year,scheduled_start_month,curr_scheduled_year,curr_sceheduled_month,year
7531,Daytona International Speedway Solar,FL,2015,12,2015.0,12.0,2014
7532,FIU Solar,FL,2015,12,2015.0,12.0,2014
8564,"Adelanto Solar, LLC",CA,2015,9,2015.0,9.0,2014
8565,"Adelanto Solar II, LLC",CA,2015,5,2015.0,7.0,2014
8670,Silver State Solar Power South,NV,2015,10,2015.0,10.0,2014
8672,Silver State Solar Power South,NV,2015,11,2015.0,11.0,2014
8673,Silver State Solar Power South,NV,2015,12,2015.0,12.0,2014
7530,Babcock Solar Energy Center,FL,2016,11,2016.0,11.0,2014
9831,"Live Oak Solar, LLC",GA,2016.0,11.0,2016.0,11.0,2015
9832,"White Pine Solar, LLC",GA,2016.0,12.0,2016.0,12.0,2015


In [26]:
solar[(solar['parent_1']==95) | (solar['parent_2']==95)].sort_values(by='plant_name')

,utility_id,utility_name,plant_code,plant_name,plant_state,county,generator_id,technology,prime_mover,unit_code,ownership,duct_burners,can_bypass_heat_recovery_steam_generator,rto/iso_lmp_node_designation,rto/iso_location_designation_for_reporting_wholesale_sales_data_to_ferc,nameplate_capacity_mw,nameplate_power_factor,summer_capacity_mw,winter_capacity_mw,minimum_load_mw,uprate_or_derate_completed_during_year,month_uprate_or_derate_completed,year_uprate_or_derate_completed,status,synchronized_to_transmission_grid,operating_month,operating_year,planned_retirement_month,planned_retirement_year,associated_with_combined_heat_and_power_system,sector_name,sector,topping_or_bottoming,energy_source_1,energy_source_2,energy_source_3,energy_source_4,energy_source_5,energy_source_6,startup_source_1,startup_source_2,startup_source_3,startup_source_4,solid_fuel_gasification_system,carbon_capture_technology,turbines_or_hydrokinetic_buoys,time_from_cold_shutdown_to_full_load,fluidized_bed_technology,pulverized_coal_technology,stoker_technology,...,planned_derate_year,planned_new_prime_mover,planned_energy_source_1,planned_new_nameplate_capacity_mw,planned_repower_month,planned_repower_year,other_planned_modifications,other_modifications_month,other_modifications_year,multiple_fuels,cofire_fuels,switch_between_oil_and_natural_gas,street_address,city,state,zip,owner_of_plants_reported_on_form,operator_of_plants_reported_on_form,asset_manager_of_plants_reported_on_form,other_relationships_with_plants_reported_on_form,entity_type,joint_owner,single_owner,other_owner,owner_name,owner_street_address,owner_city,owner_state,owner_zip,ownership_id,percent_owned,perc_owned,num_owners,idx,owner_1,owner_2,owner_3,owner_4,perc_owner_1,perc_owner_2,perc_owner_3,perc_owner_4,parent_1,parent_2,parent_3,parent_4,diff_parent,nameplate_cap,parent_1_name,parent_2_name
19063,59212,"Adelanto Solar II, LLC",59440.0,"Adelanto Solar II, LLC",CA,San Bernardino,SAS,Solar Photovoltaic,PV,NaN,S,X,X,VICTOR_7_N011,NaN,7,,7.0,7.0,,N,,,OP,X,7.0,2015.0,,,N,IPP Non-CHP,2.0,X,SUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,,NaN,NaN,NaN,NaN,...,,NaN,NaN,,,,NaN,,,N,NaN,NaN,700 Universe Blvd,Juno Beach,FL,33408,Y,0,0,0,Q,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0,NaN,NaN,NaN,NaN,7.0,nextera,NaN
19064,59211,"Adelanto Solar, LLC",59441.0,"Adelanto Solar, LLC",CA,San Bernardino,SAS,Solar Photovoltaic,PV,NaN,S,X,X,NaN,NaN,20,,20.0,20.0,,N,,,OP,X,9.0,2015.0,,,N,IPP Non-CHP,2.0,X,SUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,,NaN,NaN,NaN,NaN,...,,NaN,NaN,,,,NaN,,,N,NaN,NaN,700 Universe Blvd,Juno Beach,FL,33408,Y,0,0,0,Q,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0,NaN,NaN,NaN,NaN,20.0,nextera,NaN
19668,6452,Florida Power & Light Co,59993.0,Babcock Solar Energy Center,FL,Charlotte,1,Solar Photovoltaic,PV,NaN,S,X,X,NaN,NaN,74.5,,74.9,74.9,,N,,,OP,X,12.0,2016.0,,,N,Electric Utility,1.0,X,SUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,...,,NaN,NaN,,,,NaN,,,N,NaN,NaN,P O Box 29100,Miami,FL,33102,Y,0,0,0,I,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0,NaN,NaN,NaN,NaN,74.5,nextera,NaN
20726,6452,Florida Power & Light Co,61051.0,Barefoot Bay Solar Energy Center,FL,Brevard,1,Solar Photovoltaic,PV,NaN,S,X,X,NaN,NaN,74.5,,74.9,74.9,,N,,,OP,X,3.0,2018.0,,,N,Electric Utility,1.0,X,SUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,...,,NaN,NaN,,,,NaN,,,N,NaN,NaN,P O Box 29100,Miami,FL,33102,Y,0,0,0,I,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0,NaN,NaN,NaN,NaN,74.5,nextera,NaN
20137,60250,"DG Bethlehem Solar, LLC",60477.0,Bethlehem - East,NY,Albany,BE,Solar Photovoltaic,PV,NaN,S,X,X,NaN,NaN,1.7,,1.7,1.7,,N,,,OP,X,3.0,2016.0,,,N,IPP Non-CHP,2.0,X,SUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,...,,NaN,NaN,,,,NaN,,,NaN,NaN,NaN,700 Universe Blvd,Juno Beach,FL,33408,Y,0,0,0,Q,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,N

## answer ga solar questions


In [25]:
ga_solar = solar[solar['plant_state']=='GA']


In [35]:
ga_solar[ga_solar['parent_1'] == 'southern company']

,utility_id,utility_name,plant_code,plant_name,plant_state,county,generator_id,technology,prime_mover,unit_code,ownership,duct_burners,can_bypass_heat_recovery_steam_generator,rto/iso_lmp_node_designation,rto/iso_location_designation_for_reporting_wholesale_sales_data_to_ferc,nameplate_capacity_mw,nameplate_power_factor,summer_capacity_mw,winter_capacity_mw,minimum_load_mw,uprate_or_derate_completed_during_year,month_uprate_or_derate_completed,year_uprate_or_derate_completed,status,synchronized_to_transmission_grid,operating_month,operating_year,planned_retirement_month,planned_retirement_year,associated_with_combined_heat_and_power_system,sector_name,sector,topping_or_bottoming,energy_source_1,energy_source_2,energy_source_3,energy_source_4,energy_source_5,energy_source_6,startup_source_1,startup_source_2,startup_source_3,startup_source_4,solid_fuel_gasification_system,carbon_capture_technology,turbines_or_hydrokinetic_buoys,time_from_cold_shutdown_to_full_load,fluidized_bed_technology,pulverized_coal_technology,stoker_technology,...,planned_net_winter_capacity_derate_mw,planned_derate_month,planned_derate_year,planned_new_prime_mover,planned_energy_source_1,planned_new_nameplate_capacity_mw,planned_repower_month,planned_repower_year,other_planned_modifications,other_modifications_month,other_modifications_year,multiple_fuels,cofire_fuels,switch_between_oil_and_natural_gas,street_address,city,state,zip,owner_of_plants_reported_on_form,operator_of_plants_reported_on_form,asset_manager_of_plants_reported_on_form,other_relationships_with_plants_reported_on_form,entity_type,joint_owner,single_owner,other_owner,owner_name,owner_street_address,owner_city,owner_state,owner_zip,ownership_id,percent_owned,perc_owned,num_owners,idx,owner_1,owner_2,owner_3,owner_4,perc_owner_1,perc_owner_2,perc_owner_3,perc_owner_4,parent_1,parent_2,parent_3,parent_4,diff_parent,nameplate_cap
19075,17650,Southern Power Co,59449.0,Decatur County Solar Project,GA,Decatur,DCSP1,Solar Photovoltaic,PV,NaN,S,X,X,NaN,NaN,20,,19.0,19.0,,N,,,OP,X,12.0,2015.0,,,N,IPP Non-CHP,2.0,X,SUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,...,,,,NaN,NaN,,,,NaN,,,N,NaN,NaN,"30 Ivan Allen Jr. Blvd, NW",Atlanta,GA,30308,Y,0,0,0,I,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,southern company,NaN,NaN,NaN,NaN,20.0
19076,17650,Southern Power Co,59450.0,"Decatur Parkway Solar Project, LLC",GA,Decatur,DPSP1,Solar Photovoltaic,PV,NaN,S,X,X,NaN,NaN,80,,80.0,80.0,,N,,,OP,X,12.0,2015.0,,,N,IPP Non-CHP,2.0,X,SUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,...,,,,NaN,NaN,,,,NaN,,,N,NaN,NaN,"30 Ivan Allen Jr. Blvd, NW",Atlanta,GA,30308,Y,0,0,0,I,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,southern company,NaN,NaN,NaN,NaN,80.0
19185,7140,Georgia Power Co,59556.0,Dalton 2,GA,Murray,1,Solar Photovoltaic,PV,NaN,S,X,X,NaN,NaN,6.5,,6.5,6.5,,N,,,OP,X,12.0,2014.0,,,N,Electric Utility,1.0,X,SUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,...,,,,NaN,NaN,,,,NaN,,,N,NaN,NaN,P O Box 4545,Atlanta,GA,30302,Y,0,0,0,I,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,southern company,NaN,NaN,NaN,NaN,6.5
19552,7140,Georgia Power Co,59862.0,Fort Benning Solar Facility,GA,Chattahoochee,1,Solar Photovoltaic,PV,NaN,S,X,X,NaN,NaN,30,,30.0,30.0,,N,,,OP,X,12.0,2015.0,,,N,Electric Utility,1.0,X,SUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,...,,,,NaN,NaN,,,,NaN,,,N,NaN,NaN,P O Box 4545,Atlanta,GA,30302,Y,0,0,0,I,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,southern company,NaN,NaN,NaN,NaN,30.0
19553,7140,Georgia Power Co,59863.0,Fort Gordon Solar Facility,GA,Richmond,1,Solar Photovoltaic,PV,NaN,S,X,X,NaN,NaN,30,,30.0,30.0,,N,,,OP,X,10.0,2016.0,,,N,Electric Utility,1.0,X,SUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,...,,,,NaN,NaN,,,,NaN,,,N,NaN,NaN,P O Box 4545,Atlanta,GA,30302,Y,0,0,0,I,0,1,0,NaN,NaN,